# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset



# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')



quick-starts-ws-135106
aml-quickstarts-135106
southcentralus
610d6e37-4747-4a20-80eb-3aad70a55f43


In [4]:
# choose a name for experiment
experiment_name = 'capstone-Automl'
project_folder = './pipeline-project'

experiment=Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
capstone-Automl,quick-starts-ws-135106,Link to Azure Machine Learning studio,Link to Documentation


# Creat Compute Cluster

In [5]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException


# Choose a name for your GPU cluster
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().
compute_target.get_status()

Creating
Succeeded................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


Data

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory


In [17]:
datastore= ws.get_default_datastore()

In [18]:
datastore.upload(src_dir= './', target_path= './')

Uploading an estimated of 5 files
Target already exists. Skipping upload for automl.ipynb
Target already exists. Skipping upload for heart_failure_clinical_records_dataset.csv
Target already exists. Skipping upload for hyperparameter_tuning.ipynb
Target already exists. Skipping upload for .ipynb_checkpoints/automl-checkpoint.ipynb
Target already exists. Skipping upload for .ipynb_checkpoints/hyperparameter_tuning-checkpoint.ipynb
Uploaded 0 files


$AZUREML_DATAREFERENCE_workspaceblobstore

In [19]:
tds= Dataset.Tabular.from_delimited_files(datastore.path('./heart_failure_clinical_records_dataset.csv'))

In [20]:
tds.to_pandas_dataframe().head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [23]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'AUC_weighted'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=tds,
                             label_column_name="DEATH_EVENT",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [24]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [25]:
# Retrieve and save your best automl model.
remote_run.wait_for_completion(show_output=True)




Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: 

AttributeError: Can't get attribute 'XGBoostLabelEncoder' on <module 'xgboost.compat' from '/anaconda/envs/azureml_py36/lib/python3.6/site-packages/xgboost/compat.py'>

In [26]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [30]:
#TODO: Save the best model
best_run, fitted_model = remote_run.get_output()


AttributeError: Can't get attribute 'XGBoostLabelEncoder' on <module 'xgboost.compat' from '/anaconda/envs/azureml_py36/lib/python3.6/site-packages/xgboost/compat.py'>

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [32]:
description = 'HearFailure AutoML Model'
model = remote_run.register_model(description = description,
                               tags={'area': 'mnist'})

print(remote_run.model_id)

AutoML8e0cfeea961


TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service